## 2010-2019 kospi200 편입여부 확인

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv('./data/crawling/preprocessed/Lv.1_dongchan.csv',encoding='euc-kr',index_col=0)
df = df.drop(['str_tokens','url'],axis=1)
df_boolean = pd.read_csv('./data/kospi/raw/KOSPI200_편입여부1.csv',low_memory=False)

In [3]:
def length_adjust(str_) : 
    return (2-len(str_)) * '0' + str_

def date_convertor(date) : 
    year = length_adjust(str(date.year))
    month = length_adjust(str(date.month))
    day = length_adjust(str(date.day))
    yyyymmdd = int(year+month+day)
    return yyyymmdd

In [4]:
dts = df_boolean.iloc[1:,0] #datetime형식을 통일시켜준다
dates_str = []
for idx,i in enumerate(dts):
    i = df_boolean.iloc[idx+1,0][0:4]+df_boolean.iloc[idx+1,0][5:7]+df_boolean.iloc[idx+1,0][8:10] 
    dates_str.append(i)

dates_str.insert(0,'Symbol Name')


symbol_ls = [x for x in df_boolean.columns][1:]
crp_ls = []
for i in symbol_ls:
    if df_boolean[i][1:].isin(['Y']).all() == True:   #전체 kospi기업에서 10년동안 kospi200에 편입되었던 기업들만 추출
        crp_ls.append(i)


dts = df_boolean.iloc[1:,0] #datetime형식을 통일시켜준다
dates_str = []
for idx,i in enumerate(dts):
    i = df_boolean.iloc[idx+1,0][0:4]+df_boolean.iloc[idx+1,0][5:7]+df_boolean.iloc[idx+1,0][8:10] 
    dates_str.append(i)

dates_str.insert(0,'Symbol Name')

df_boolean = df_boolean.loc[:,crp_ls]
df_boolean.index = dates_str


In [5]:
df_boolean.head()

,A000070,A000100,A000120,A000150,A000210,A000240,A000270,A000640,A000660,A000670,...,A069960,A071050,A078930,A086790,A090430,A093050,A093370,A096770,A097950,A105560
Symbol Name,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20100101,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20100102,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20100103,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20100104,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y


## kospi200 기업의 일별 종가 추출

FinanceDataReader 패키지를 이용하여 2010년부터 2019년 말까지 해당 기업의 종가를 가져오는 함수를 구현한다. 개별기업의 주가를 dataframe형식으로 반환하므로 10년동안 지수에 편입되었던 113개의 기업들에 대하여 수정종가, 수익률 , 그리고 naive하게 market return을 제외한 수익률을 구하였다.

In [6]:
def close_price(code): #개별기업코드 입력시 종가 반환
    
    import FinanceDataReader as fdr
    
    df = fdr.DataReader(code, start = '2010-01-01' , end = '2019-12-31')
    df = df['Close']
    
    return df

In [7]:
def get_return(code): #개별기업코드 입력시 수익률 반환
    
    import FinanceDataReader as fdr
    
    df = fdr.DataReader(code, start = '2010-01-01' , end = '2019-12-31')
    df = df['Change']
    
    return df

In [8]:
df_boolean

,A000070,A000100,A000120,A000150,A000210,A000240,A000270,A000640,A000660,A000670,...,A069960,A071050,A078930,A086790,A090430,A093050,A093370,A096770,A097950,A105560
Symbol Name,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20100101,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20100102,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20100103,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20100104,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191227,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20191228,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20191229,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20191230,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y


In [ ]:
symbols = [x for x in df_boolean.iloc[0,:]] #기업명 리스트
t_ls = [x[1:] for x in df_boolean.columns] #종목코드
symbol_dic = {name:value for name,value in zip(symbols,t_ls)}


close_ls = [close_price(x) for x in t_ls] 
return_ls = [get_return(x) for x in t_ls]

df__price = pd.concat([i for i in close_ls],ignore_index=False, axis =1)
df__price.columns = symbols
# df__price = df__price.fillna(int(0)) #str to int.. NaN값은0으로 변환

df_return = pd.concat([i for i in return_ls],ignore_index=False, axis =1)
df_return.columns = symbols
# df_return = df_return.fillna(int(0)) #str to int.. NaN값은0으로 변환


#datetime string으로 변환
dates = df__price.index
dates_ls = list(map(date_convertor,dates)) #datetime string으로 변환
df_price = df__price.copy()

df_price.index = dates_ls
df_return.index = dates_ls

In [ ]:
df_return.shape
nan_mean  = np.nanmean(df_return, axis=1) #nan값빼고 market return을 naive하게 계산
demean = df_return.values-nan_mean.reshape(len(nan_mean),1) #market return 제외

df_demean= pd.DataFrame(demean,index=df_return.index, columns = df_return.columns)

In [ ]:
df_price.head()

In [ ]:
df_return.head()

In [ ]:
df_demean.head()

## crawling data 전처리

In [ ]:
def preprocessing(document): #특수문자 기본 전처리 함수
    docu = []
    for doc in document:
        doc = re.sub('[^가-힣0-9\s.]','',doc)
        doc = re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',doc)
#         doc = re.sub('[0-9]+','num',doc) #숫자를 special token num으로 치환
        pattern = re.compile(r'\s+') #중복띄어쓰기 제거
        doc = re.sub(pattern,' ',doc)
        doc = doc.replace('\xa0','') 
        doc = doc.replace('\n','')
        docu.append(doc)

    return docu

In [ ]:
nan_idx = [x for x in df[df.str.isnull() ==True].index]
df = df.drop(nan_idx,axis=0) #str 결측치 삭제
df.rmk = df.rmk.fillna('nan')
df = df.reset_index(drop=True)

str_ls = [x for x in df['str']]
corrected_ls = preprocessing(str_ls)
df['str'] = corrected_ls

크롤링한 LV1 data(kospi200 한 번이라도 편입되었던 309개 기업의 텍스트)에 대한 토큰화 이전 기본 전처리를 하고 결측치 처리해주었다.

종목코드는 같지만 거래소에 공시되어 있는 기업의 이름과 dart에서 크롤링한 기업의 이름이 상충되는 것이 있어서 이를 거래소 이름으로 통일하여 주었다. 여기서 지수 편입 퇴출 효과를 제거하기 위해 10년동안 한번도 퇴출된 적이 없었던 113개의 기업들로 데이터를 다시 구성하였다.

In [ ]:
change_dic = {'케이씨씨':'KCC',
 '케이티':'KT',
'케이티앤지':'KT&G',
'포스코':'POSCO',
'금호석유화학':'금호석유',
 '기아자동차':'기아차',
'롯데칠성음료':'롯데칠성',
 '삼성화재해상보험':'삼성화재',
 '쌍용양회공업': '쌍용양회',
'아모레퍼시픽그룹':'아모레G',
 '한국단자공업':'한국단자',
'한국전력공사':'한국전력',
'현대엘리베이터':'현대엘리베이',
 '현대자동차':'현대차'}


new_crp_ls = []
for i in df.crp_nm:
    for x in change_dic.keys():
        if i == x:
            i = change_dic[x]
    new_crp_ls.append(i)

df.crp_nm = new_crp_ls

symbols = df_boolean.iloc[0,:] 
con_ls = [df[df.crp_nm == i] for i in symbols]
df_all = pd.concat(x for x in con_ls).reset_index(drop=True) #10년동안 편입되어있던 113개의 기업

In [ ]:
df_all.head(10)